<a href="https://colab.research.google.com/github/hhdjwdabsxsx/Sentiment-Analysis-with-Recurrent-Neural-Networks/blob/main/Copy_of_Sentiment_Analysis_with_Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sentiment Analysis with Recurrent Neural Networks  **

**Importing Libraries and Dataset**

In [ ]:
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
import numpy as np

We will be using Keras IMDB dataset. Vocabulary size is a parameter that is used the get data containing the given number of most occurring words in the entire corpus of textual size.

In [ ]:
# Getting reviews with words that come under 5000
# most occurring words in the entire
# corpus of textual review data
vocab_size = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

print(x_train[0])

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


These are the index values of the words and hence we done see any reviews.

In [ ]:
# Getting all the words from word_index dictionary
word_idx = imdb.get_word_index()

# Originally the index number of a value and not a key,
# hance converting the index as key snd the words as values
word_idx = {i: word for word, i in word_idx.items()}

# again printing the review
print([word_idx[i] for i in x_train[0]])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when',

Let's check the range of the reviews we have in this dataset.

In [ ]:
# Get the minimum and the maximum length of reviews
print("Max length of a review:: ", len(max((x_train+x_test), key=len)))
print("Min length of a review:: ", len(min((x_train+x_test), key=len)))

Max length of a review::  2697
Min length of a review::  70


We see that the longest review available is 2697 words and the shortest one is 70. While working the Neural Netwroks, it is important to make all the inputs in a fixed size. To achieve this objective we will pad the review sentences.

In [ ]:
from tensorflow.keras.preprocessing import sequence

# Keeping a fixed length of all reviews to max 400 words
max_words = 400

x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

x_valid, y_valid = x_train[:64], y_train[:64]
x_train_, y_train_ = x_train[64:], y_train[64:]



**SimpleRNN (also called Vanilla RNN)**

They are the most basic form of Recurrent Neural Networks that trie to memorize sequential information. However, they have the native problems of Exploding and Vanishing gradients.

In [ ]:
# fixing every word's embedding size to be 32
embd_len = 32

# Creating a RNN model
RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(vocab_size,
                        embd_len,
                        input_length=max_words))

# In case of a stacked(more than one layer of RNN)
# use return_sequences=True
RNN_model.add(SimpleRNN(128,
                        activation='tanh',
                        return_sequences=False))
RNN_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(RNN_model.summary())

# Compiling model
RNN_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

# Training the model
history = RNN_model.fit(x_train_, y_train_,
                        batch_size=64,
                        epochs=5,
                        verbose=1,
                        validation_data=(x_valid, y_valid))

# Printing model score on test data
print()
print("Simple_RNN Score---> ", RNN_model.evaluate(x_test, y_test, verbose=0))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "Simple_RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 79s 195ms/step - accuracy: 0.5293 - loss: 0.6884 - val_accuracy: 0.5938 - val_loss: 0.6714
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 84s 201ms/step - accuracy: 0.6497 - loss: 0.6231 - val_accuracy: 0.6094 - val_loss: 0.6329
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 75s 193ms/step - accuracy: 0.7235 - loss: 0.5453 - val_accuracy: 0.6250 - val_loss: 0.7086
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 84s 198ms/step - accuracy: 0.7572 - loss: 0.4985 - val_accuracy: 0.7500 - val_loss: 0.5670
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 80s 193ms/step - accuracy: 0.8048 - loss: 0.4350 - val_accuracy: 0.6562 - val_loss: 0.6302

Simple_RNN Score--->  [0.5708127617835999, 0.709559977054596]


The vanilla form of RNN gave us a Test Accuracy of 70.95%. Limitations of Simple RNN are it is unable to handle long sentences well because of its vanishing gradient problems.

**Gated Recurrent Units(GRU)**

GRUs are lesser known but equaally robust algorithms to solve the limitations of Simple RNNs.

In [ ]:
# Deefining GRU model
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocab_size,
                        embd_len,
                        input_length=max_words))
gru_model.add(GRU(128,
                  activation='tanh',
                  return_sequences=False))
gru_model.add(Dense(1, activation='sigmoid'))

# Printing model summary
print(gru_model.summary())

# Compiling the model
gru_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

# Training the GRU model
history2 = gru_model.fit(x_train_, y_train_,
                         batch_size=64,
                         epochs=5,
                         verbose=1,
                         validation_data=(x_valid, y_valid))

# Printing model score on test data
print()
print("GRU Score---> ", gru_model.evaluate(x_test, y_test, verbose=0))

Model: "GRU_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 323s 816ms/step - accuracy: 0.6705 - loss: 0.5649 - val_accuracy: 0.8438 - val_loss: 0.2935
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 326s 829ms/step - accuracy: 0.8689 - loss: 0.3240 - val_accuracy: 0.9375 - val_loss: 0.2173
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 327s 839ms/step - accuracy: 0.8983 - loss: 0.2540 - val_accuracy: 0.8906 - val_loss: 0.2672
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 390s 859ms/step - accuracy: 0.9189 - loss: 0.2095 - val_accuracy: 0.9062 - val_loss: 0.1959
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 369s 827ms/step - accuracy: 0.9387 - loss: 0.1649 - val_accuracy: 0.8750 - val_loss: 0.2879

GRU Score--->  [0.29506370425224304, 0.8872799873352051]


Test accuracy of GRU is found to be 88.72%. GRU is a form of RNN that are better than simple RNN and are often faster than LSTM due to its relatively fewer training parameters.

**Long Short Term Memory**

LSTM is better in terms of capturing the memory of sequential information better than simple RNNs. Due to increased complexity than that of GRU, it is slower to train but in general, LSTM give better accyuracy than GRUs.


In [ ]:
# Defining LSTM model
lstm_model = Sequential(name="LSTM_Model")
# fixing every word's embedding size to be 32
embd_len = 32 #Define embd_len here before using it
lstm_model.add(Embedding(vocab_size,
						  embd_len,
						  input_length=max_words))
lstm_model.add(LSTM(128,
					activation='relu',
					return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))

# Printing Model Summary
print(lstm_model.summary())

# Compiling the model
lstm_model.compile(
	loss="binary_crossentropy",
	optimizer='adam',
	metrics=['accuracy']
)

# Training the model
history3 = lstm_model.fit(x_train_, y_train_,
						batch_size=64,
						epochs=5,
						verbose=2,
						validation_data=(x_valid, y_valid))

# Displaying the model accuracy on test data
print()
print("LSTM model Score---> ", lstm_model.evaluate(x_test, y_test, verbose=0))

Model: "LSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 - 310s - 795ms/step - accuracy: 0.5011 - loss: nan - val_accuracy: 0.6094 - val_loss: nan
Epoch 2/5
390/390 - 322s - 825ms/step - accuracy: 0.4997 - loss: nan - val_accuracy: 0.6094 - val_loss: nan
Epoch 3/5
390/390 - 322s - 825ms/step - accuracy: 0.4997 - loss: nan - val_accuracy: 0.6094 - val_loss: nan
Epoch 4/5
390/390 - 317s - 813ms/step - accuracy: 0.4997 - loss: nan - val_accuracy: 0.6094 - val_loss: nan
Epoch 5/5
390/390 - 327s - 839ms/step - accuracy: 0.4997 - loss: nan - val_accuracy: 0.6094 - val_loss: nan

LSTM model Score--->  [nan, 0.5]


**Bi-directional LSTM Model**

Bidirectional LSTMs are a deruvative of traditional LSTMs. Here, two LSTMs are used to capture both the forward and backward sequences of the input. This helps in capturing the context better than nirmal LSTM.

In [ ]:
# Defining Bidirectional LSTM model
bi_lstm_model = Sequential(name="Bidirectional_LSTM")
bi_lstm_model.add(Embedding(vocab_size,
                            embd_len,
                            input_length=max_words))
bi_lstm_model.add(Bidirectional(LSTM(128,
                                    activation='tanh',
                                     return_sequences=False)))
bi_lstm_model.add(Dense(1, activation='sigmoid'))

# Printing model summary
print(bi_lstm_model.summary())

# Compiling model summary
bi_lstm_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

# Traininng the model
history4 = bi_lstm_model.fit(x_train_, y_train_,
                             batch_size=64,
                             epochs=5,
                             verbose=2,
                             validation_data=(x_test, y_test))

# Printing mofel score on test data
print()
print("Bidirectional LSTM model Score---> ", bi_lstm_model.evaluate(x_test, y_test, verbose=0))

Model: "Bidirectional_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 - 681s - 2s/step - accuracy: 0.7393 - loss: 0.5130 - val_accuracy: 0.8248 - val_loss: 0.4238
Epoch 2/5
390/390 - 661s - 2s/step - accuracy: 0.8606 - loss: 0.3395 - val_accuracy: 0.8596 - val_loss: 0.3366
Epoch 3/5
390/390 - 649s - 2s/step - accuracy: 0.8968 - loss: 0.2631 - val_accuracy: 0.8775 - val_loss: 0.2922
Epoch 4/5
390/390 - 663s - 2s/step - accuracy: 0.9059 - loss: 0.2428 - val_accuracy: 0.8727 - val_loss: 0.3005
Epoch 5/5
390/390 - 640s - 2s/step - accuracy: 0.9141 - loss: 0.2248 - val_accuracy: 0.8356 - val_loss: 0.3745

Bidirectional LSTM model Score--->  [0.3745136260986328, 0.8355600237846375]


In [ ]:
# Defining LSTM model
lstm_model = Sequential(name="LSTM_Model")

# Save the model in the SavedModel format
# Added the .keras extension to the filename
lstm_model.save('sentiment_analysis_model.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)
